## todo:
* * _Catagorical Dummy Encoding?__
    * Anything here? Check for catagorical using unique


# Data Cleaning Notebook


In [27]:
# Import the required libraries
import pandas as pd
import numpy as np

# set up pandas to display floats in a more human friendly way
pd.options.display.float_format = '{:,.2f}'.format

In [28]:
king_data_raw = pd.read_csv('../data/kc_house_data.csv')
king_data_raw.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,nan,0.00,...,7,1180,0.0,1955,0.00,98178,47.51,-122.26,1340,5650
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0.00,0.00,...,7,2170,400.0,1951,"1,991.00",98125,47.72,-122.32,1690,7639
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0.00,0.00,...,6,770,0.0,1933,nan,98028,47.74,-122.23,2720,8062
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0.00,0.00,...,7,1050,910.0,1965,0.00,98136,47.52,-122.39,1360,5000
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0.00,0.00,...,8,1680,0.0,1987,0.00,98074,47.62,-122.05,1800,7503
5,7237550310,5/12/2014,"1,230,000.00",4,4.50,5420,101930,1.00,0.00,0.00,...,11,3890,1530.0,2001,0.00,98053,47.66,-122.00,4760,101930
6,1321400060,6/27/2014,"257,500.00",3,2.25,1715,6819,2.00,0.00,0.00,...,7,1715,?,1995,0.00,98003,47.31,-122.33,2238,6819
7,2008000270,1/15/2015,"291,850.00",3,1.50,1060,9711,1.00,0.00,nan,...,7,1060,0.0,1963,0.00,98198,47.41,-122.31,1650,9711
8,2414600126,4/15/2015,"229,500.00",3,1.00,1780,7470,1.00,0.00,0.00,...,7,1050,730.0,1960,0.00,98146,47.51,-122.34,1780,8113
9,3793500160,3/12/2015,"323,000.00",3,2.50,1890,6560,2.00,0.00,0.00,...,7,1890,0.0,2003,0.00,98038,47.37,-122.03,2390,7570


In [29]:
king_data_raw.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float

In [30]:
# Make a deep copy and do procesing on this dataframe moving forward
# Deep copy has own copy of data and index.
king_processed = king_data_raw.copy(deep=True)

# Data Preparation

## Data cleaning steps:
* __Duplicate check__ 
    * no duplicates found
* __Address nans__
    * waterfront 
    * view
    * yr_renovated 
* __Data type conversions__
    * sqft_basement - convert from string to int
    * yr_renovated - convert from float to int
* __New column creation__
    * date - split out the Month, Day, Year for potential feature use
    * yr_renovated - create a new catagorical variable,renovated_status, based off logic detailed in section below.
    * sqft_living, sqft_living15, sqft_lot and sqft_lot15 - create two new catagorical variables, sqft_living15_different and sqft_lot15_different. More details below on reason.
* _Catagorical Dummy Encoding?__
    * Anything here? Check for catagorical using unique


## Duplicate check

In [31]:
# Duplicate check
king_dups = king_data_raw[king_data_raw.duplicated()]
print(len(king_dups))

0


## Address nans

In [32]:
# Address nans for waterfront column
print(king_data_raw.waterfront.unique())
print(king_data_raw['waterfront'].isna().sum())

[nan  0.  1.]
2376


In [33]:
# Assuming that, since waterfront property is desirable real estate, it would be marked as such. 
# Setting waterfront nan values to 0 
king_processed['waterfront'].fillna(0.0, inplace=True)
print(king_processed['waterfront'].isna().sum())

0


In [34]:
# Address nans for view column 
print(king_data_raw.view.unique())
print(king_data_raw['view'].isna().sum())

[ 0. nan  3.  4.  2.  1.]
63


In [35]:
# Assuming that, since having a high rated view attribute would indicate desirable real estate, a good view would be marked as such. 
# Setting view nan values to 0 
king_processed['view'].fillna(0.0, inplace=True)
print(king_processed['view'].isna().sum())

0


In [36]:
# Address nans for yr_renovated column
print(king_data_raw.yr_renovated.unique())
print(king_data_raw['yr_renovated'].isna().sum())

[   0. 1991.   nan 2002. 2010. 1992. 2013. 1994. 1978. 2005. 2003. 1984.
 1954. 2014. 2011. 1983. 1945. 1990. 1988. 1977. 1981. 1995. 2000. 1999.
 1998. 1970. 1989. 2004. 1986. 2007. 1987. 2006. 1985. 2001. 1980. 1971.
 1979. 1997. 1950. 1969. 1948. 2009. 2015. 1974. 2008. 1968. 2012. 1963.
 1951. 1962. 1953. 1993. 1996. 1955. 1982. 1956. 1940. 1976. 1946. 1975.
 1964. 1973. 1957. 1959. 1960. 1967. 1965. 1934. 1972. 1944. 1958.]
3842


__yr_renovated__ column/feature feature needs some more thought applied to how to handle it. 
I'll start by setting the nans to 0.  
I might need to add a renovated_status categorical variable.
Here's my thoughts on some valid assumptions around renovated_status.
* Recently constructed homes have probably not been renovated, so even if the __yr_renovated__ was nan/0, I will assume home wasn't renovated.
* __Also worth noting__ that if there is a difference, I might need to use the "2015" columns/features instead of the "original" features.  Could do two different models/split out the data.




In [37]:
# Setting the nans to 0. Will perform other checks to set a new categorical variable, renovated_status, later on.
king_processed['yr_renovated'].fillna(0, inplace=True)
print(king_processed['yr_renovated'].isna().sum())

0


## Data type conversions

In [38]:
#no issues with simply changing yr_renovated to int64
king_processed['yr_renovated'] = np.int64(king_processed['yr_renovated'])

In [39]:
# sqft_basement needed more wrangling 
# Was a String with either a non-numeric value (like ?) or a String representation of a float 400.00 with unneeded decimal places :/
def convert_sqft_basement(some_string, default):
    result = np.int64(default)
    try:
        result = np.int64(float(some_string))
    except (ValueError, TypeError):
        print(f'could not convert {some_string}, using {result}')
    finally:
        return result

king_processed['sqft_basement'] = king_processed.apply(lambda row: convert_sqft_basement(row.sqft_basement, 0), axis=1)

could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not conv

In [40]:
print(king_processed.dtypes)
king_processed.head(10)

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront       float64
view             float64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,0.00,0.00,...,7,1180,0,1955,0,98178,47.51,-122.26,1340,5650
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0.00,0.00,...,7,2170,400,1951,1991,98125,47.72,-122.32,1690,7639
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0.00,0.00,...,6,770,0,1933,0,98028,47.74,-122.23,2720,8062
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0.00,0.00,...,7,1050,910,1965,0,98136,47.52,-122.39,1360,5000
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0.00,0.00,...,8,1680,0,1987,0,98074,47.62,-122.05,1800,7503
5,7237550310,5/12/2014,"1,230,000.00",4,4.50,5420,101930,1.00,0.00,0.00,...,11,3890,1530,2001,0,98053,47.66,-122.00,4760,101930
6,1321400060,6/27/2014,"257,500.00",3,2.25,1715,6819,2.00,0.00,0.00,...,7,1715,0,1995,0,98003,47.31,-122.33,2238,6819
7,2008000270,1/15/2015,"291,850.00",3,1.50,1060,9711,1.00,0.00,0.00,...,7,1060,0,1963,0,98198,47.41,-122.31,1650,9711
8,2414600126,4/15/2015,"229,500.00",3,1.00,1780,7470,1.00,0.00,0.00,...,7,1050,730,1960,0,98146,47.51,-122.34,1780,8113
9,3793500160,3/12/2015,"323,000.00",3,2.50,1890,6560,2.00,0.00,0.00,...,7,1890,0,2003,0,98038,47.37,-122.03,2390,7570


## New columns creation

In [41]:
# The date column is not really useful. Pull out the month, day, year values from this date String column and save a new cols.
def get_date_element(date_as_string, desired_index):
    split_vals = date_as_string.split('/')
    if len(split_vals) != 3:
        split_vals = [0, 0, 0]
    return np.int64(split_vals[desired_index])


In [42]:
king_processed['date_month'] = king_processed.apply(lambda row: get_date_element(row.date, 0), axis=1)
king_processed['date_day'] = king_processed.apply(lambda row: get_date_element(row.date, 1), axis=1)
king_processed['date_year'] = king_processed.apply(lambda row: get_date_element(row.date, 2), axis=1)

In [43]:
# Make a categorical variable to reflect the renovated status. 
# Start by setting value Y if there is a valid yr_renovated year (>0) and UNK if it is 0
king_processed['renovated_status'] = king_processed.apply(lambda row: 'Y' if row.yr_renovated > 0 else 'UNK', axis=1)


In [44]:
# Where the yr_renovated is provided, what is the most recent yr_built?
print(king_processed[(king_processed['yr_renovated']>0)].yr_built.max())

2003


For the set of homes with a valid __yr_renovated__, the most recent yr_built was 2003.  I will assume that any home build after 2003 with __yr_renovated__ of 0 was __not renovated__ and update the __renovated_status__ from 'UNK' to 'N'. 

In [45]:
print(king_processed[king_processed['renovated_status'] == 'UNK'].shape)
print(king_processed[king_processed['renovated_status'] == 'N'].shape)

(20853, 25)
(0, 25)


In [46]:
def check_for_recent_homes(yr_renovated, yr_built):
    return (yr_renovated == 0 and yr_built > 2003)

king_processed['renovated_status'] = king_processed.apply(lambda row: 'N' if check_for_recent_homes(row.yr_renovated, row.yr_built) else row.renovated_status, axis=1)
king_processed.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_month,date_day,date_year,renovated_status
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,0.00,0.00,...,0,98178,47.51,-122.26,1340,5650,10,13,2014,UNK
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0.00,0.00,...,1991,98125,47.72,-122.32,1690,7639,12,9,2014,Y
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0.00,0.00,...,0,98028,47.74,-122.23,2720,8062,2,25,2015,UNK
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0.00,0.00,...,0,98136,47.52,-122.39,1360,5000,12,9,2014,UNK
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0.00,0.00,...,0,98074,47.62,-122.05,1800,7503,2,18,2015,UNK


In [47]:
print(king_processed[king_processed['renovated_status'] == 'UNK'].shape)
print(king_processed[king_processed['renovated_status'] == 'N'].shape)

(17262, 25)
(3591, 25)


From the data description, the _15 columns contain avg sqft from 15 nearest neighbors (living area and lot size). This would indicate if a house was smaller/similar/larger than its neighbors. 
* sqft_living15 - the average house square footage of the 15 closest houses
* sqft_lot15 - the average lot square footage of the 15 closest houses

Creating create new columns to capture differences between the sqft_living and sqft_lot and their corresponding neighbors values


In [55]:
king_processed['sqft_living15_diff'] = king_processed.apply(lambda row: row.sqft_living - row.sqft_living15, axis=1)
king_processed['sqft_lot15_diff'] = king_processed.apply(lambda row: row.sqft_lot - row.sqft_lot15, axis=1)

In [56]:
king_processed.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_living15,sqft_lot15,date_month,date_day,date_year,renovated_status,sqft_living15_diff,sqft_lot15_diff,sqft_living15_different,sqft_lot15_different
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,0.00,0.00,...,1340,5650,10,13,2014,UNK,-160,0,Y,N
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0.00,0.00,...,1690,7639,12,9,2014,Y,880,-397,Y,Y
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0.00,0.00,...,2720,8062,2,25,2015,UNK,-1950,1938,Y,Y
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0.00,0.00,...,1360,5000,12,9,2014,UNK,600,0,Y,N
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0.00,0.00,...,1800,7503,2,18,2015,UNK,-120,577,Y,Y
5,7237550310,5/12/2014,"1,230,000.00",4,4.50,5420,101930,1.00,0.00,0.00,...,4760,101930,5,12,2014,UNK,660,0,Y,N
6,1321400060,6/27/2014,"257,500.00",3,2.25,1715,6819,2.00,0.00,0.00,...,2238,6819,6,27,2014,UNK,-523,0,Y,N
7,2008000270,1/15/2015,"291,850.00",3,1.50,1060,9711,1.00,0.00,0.00,...,1650,9711,1,15,2015,UNK,-590,0,Y,N
8,2414600126,4/15/2015,"229,500.00",3,1.00,1780,7470,1.00,0.00,0.00,...,1780,8113,4,15,2015,UNK,0,-643,N,Y
9,3793500160,3/12/2015,"323,000.00",3,2.50,1890,6560,2.00,0.00,0.00,...,2390,7570,3,12,2015,UNK,-500,-1010,Y,Y


## Save processed data to file


In [26]:
# save the processed data to file
king_processed.to_csv('../data/proc_kc_house_data.csv', index=False)